In [ ]:
import pandas as pd
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#github access token

headers = {
    'Authorization':'********'
}

In [ ]:
# github rate limit tracking

import datetime

response = requests.get('https://api.github.com/rate_limit', headers=headers)
rate_limit_data = response.json()

rate_limit=rate_limit_data['resources']['core']['limit']
rate_remaining=rate_limit_data['resources']['core']['remaining']
reset_timestamp = rate_limit_data['resources']['core']['reset']
reset_datetime = datetime.datetime.fromtimestamp(reset_timestamp)

print(rate_limit,rate_remaining, reset_datetime)

60 60 2024-10-30 23:07:07


In [ ]:
# import the list of users

users_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TDS/Project 1 - Basel10/users.csv')
users_list=users_df['login'].tolist()

len(users_list)

349

In [ ]:
# extract the most recent 500 repos details for each of the filtered users

repo_fields = req_fields=['login','full_name','created_at','stargazers_count','watchers_count','language','has_projects','has_wiki','license']
repo_df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TDS/Project 1 - Basel10/repos.csv')

repo_df.shape

(12836, 9)

In [ ]:
n=len(repo_df['login'].unique())
n

277

In [ ]:
#looping for each user
for user in users_list[n:n+rate_remaining-1]:

  url=f'https://api.github.com/users/{user}/repos'
  user_repo_list=[]

  #looping through each page for the user_repo results
  while url:

    response=requests.get(url,headers=headers)
    if response.status_code==200:
      user_repo_data=response.json()
      user_repo_list.extend(user_repo_data)
      link_header=response.headers.get('link')

      if link_header and 'rel="next"' in link_header:
        for link in link_header.split(','):
          if 'rel="next"' in link:
            url=link.split(';')[0].strip('<> ')
            break
          else:
            url= None
      else:
        url=None



  # pulling out the latest 500 repos into a dataframe for the current user in the loop

  sorted_repo_list=sorted(user_repo_list,key=lambda x:x['created_at'],reverse=True)
  repo_data_for_user = []

  for i in range(min(500,len(sorted_repo_list))):
    # getting the fields for the ith repo of this user
    repo_list={}
    for field in repo_fields:
      if field=='login':
        repo_list[field]=user
      elif field=='license':
        repo_list[field]=sorted_repo_list[i][field]['name'] if sorted_repo_list[i][field] else None
      else:
        repo_list[field]=sorted_repo_list[i][field]
      # appending this repo data of the repo_data_fo_user
    repo_data_for_user.append(repo_list)

  # adding repo data of this user to the main repo_df
  repo_df = pd.concat([repo_df, pd.DataFrame(repo_data_for_user)], ignore_index=True)

KeyboardInterrupt: 

In [ ]:
user


'coatpont'

In [ ]:
rdu=pd.DataFrame(repo_data_for_user)
rdu.shape

(9, 9)

In [ ]:
repo_df.shape

(13713, 9)

In [ ]:
repo_df['login'].value_counts()

,count
login,
syzer,500
dpryan79,443
aalmiray,253
galderz,232
ioolkos,218
...,...
undeadpixel,15
wasserth,14
gabrielwalt,10


In [ ]:
repo_df.to_csv('/content/drive/MyDrive/Colab Notebooks/TDS/Project 1 - Basel10/repos.csv',index=False)